https://realpython.com/pandas-groupby/

# Pandas GroupBy: Your Guide to Grouping Data in Python

In [ ]:
Table of Contents

Housekeeping
Example 1: U.S. Congress Dataset
The “Hello, World!” of Pandas GroupBy
Pandas GroupBy vs SQL
How Pandas GroupBy Works
Example 2: Air Quality Dataset
Grouping on Derived Arrays
Resampling
Example 3: News Aggregator Dataset
Using Lambda Functions in .groupby()
Improving the Performance of .groupby()
Pandas GroupBy: Putting It All Together
Conclusion
More Resources on Pandas GroupBy

In [2]:
#The examples here also use a few tweaked Pandas options for friendlier output:

import pandas as pd

# Use 3 decimal places in output display
pd.set_option("display.precision", 3)

# Don't wrap repr(DataFrame) across additional lines
pd.set_option("display.expand_frame_repr", False)

# Set max rows displayed in output to 25
pd.set_option("display.max_rows", 25)

In [ ]:
The -d option lets you extract the contents to a new folder:

./
│
└── groupby-data/
    │
    ├── legislators-historical.csv
    ├── airqual.csv
    └── news.csv

## Example 1: U.S. Congress Dataset

In [4]:
import pandas as pd

dtypes = {
    "first_name": "category",
    "gender": "category",
    "type": "category",
    "state": "category",
    "party": "category",
}
df = pd.read_csv(
    "legislators-historical.csv",
    dtype=dtypes,
    usecols=list(dtypes) + ["birthday", "last_name"],
    parse_dates=["birthday"]
)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12045 entries, 0 to 12044
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   last_name   12045 non-null  object        
 1   first_name  12045 non-null  category      
 2   birthday    11495 non-null  datetime64[ns]
 3   gender      12045 non-null  category      
 4   type        12045 non-null  category      
 5   state       12045 non-null  category      
 6   party       11813 non-null  category      
dtypes: category(5), datetime64[ns](1), object(1)
memory usage: 317.2+ KB


In [7]:
df.tail()

last_name first_name   birthday gender type state       party
12040  Riggleman     Denver 1970-03-01      M  rep    VA  Republican
12041       Hall     Kwanza 1971-05-01      M  rep    GA    Democrat
12042   Richmond     Cedric 1973-09-13      M  rep    LA    Democrat
12043     Harris     Kamala 1964-10-20      F  sen    CA    Democrat
12044   Loeffler      Kelly 1970-11-27      F  sen    GA  Republican

In [8]:
#The DataFrame uses categorical dtypes for space efficiency:

df.dtypes

last_name             object
first_name          category
birthday      datetime64[ns]
gender              category
type                category
state               category
party               category
dtype: object

You can see that most columns of the dataset have the type category, which reduces the memory load on your machine.

In [ ]:
The “Hello, World!” of Pandas GroupBy
Now that you’re familiar with the dataset, you’ll start with a “Hello, World!” for the Pandas GroupBy operation. What is the count of Congressional members, on a state-by-state basis, over the entire history of the dataset? In SQL, you could find this answer with a SELECT statement:

SELECT state, count(name)
FROM df
GROUP BY state
ORDER BY state;

In [9]:
#Here’s the near-equivalent in Pandas:

n_by_state = df.groupby("state")["last_name"].count()
n_by_state.head(10)

state
AK     16
AL    209
AR    117
AS      2
AZ     49
CA    367
CO     92
CT    240
DC      2
DE     97
Name: last_name, dtype: int64

You call .groupby() and pass the name of the column you want to group on, which is "state". Then, you use ["last_name"] to specify the columns on which you want to perform the actual aggregation.

You can pass a lot more than just a single column name to .groupby() as the first argument. You can also specify any of the following:

A list of multiple column names
A dict or Pandas Series
A NumPy array or Pandas Index, or an array-like iterable of these
Here’s an example of grouping jointly on two columns, which finds the count of Congressional members broken out by state and then by gender:

In [10]:
df.groupby(["state", "gender"])["last_name"].count()

state  gender
AK     F           0
       M          16
AL     F           4
       M         205
AR     F           5
                ... 
WI     M         198
WV     F           1
       M         119
WY     F           1
       M          39
Name: last_name, Length: 116, dtype: int64

The analogous SQL query would look like this:

SELECT state, gender, count(name)
FROM df
GROUP BY state, gender
ORDER BY state, gender;

### Pandas GroupBy vs SQL

In [ ]:
This is a good time to introduce one prominent difference between the Pandas GroupBy operation and the SQL query above. The result set of the SQL query contains three columns:

state
gender
count
In the Pandas version, the grouped-on columns are pushed into the MultiIndex of the resulting Series by default:

In [11]:
n_by_state_gender = df.groupby(["state", "gender"])["last_name"].count()
type(n_by_state_gender)

n_by_state_gender.index[:5]

MultiIndex([('AK', 'F'),
            ('AK', 'M'),
            ('AL', 'F'),
            ('AL', 'M'),
            ('AR', 'F')],
           names=['state', 'gender'])

To more closely emulate the SQL result and push the grouped-on columns back into columns in the result, you an use as_index=False:

In [12]:
df.groupby(["state", "gender"], as_index=False)["last_name"].count()

state gender  last_name
0      AK      F        NaN
1      AK      M       16.0
2      AL      F        4.0
3      AL      M      205.0
4      AR      F        5.0
..    ...    ...        ...
111    WI      M      198.0
112    WV      F        1.0
113    WV      M      119.0
114    WY      F        1.0
115    WY      M       39.0

[116 rows x 3 columns]

This produces a DataFrame with three columns and a RangeIndex, rather than a Series with a MultiIndex. In short, using as_index=False will make your result more closely mimic the default SQL output for a similar operation.

Note: In df.groupby(["state", "gender"])["last_name"].count(), you could also use .size() instead of .count(), since you know that there are no NaN last names. Using .count() excludes NaN values, while .size() includes everything, NaN or not.

Also note that the SQL queries above explicitly use ORDER BY, whereas .groupby() does not. That’s because .groupby() does this by default through its parameter sort, which is True unless you tell it otherwise:

In [13]:
# Don't sort results by the sort keys
df.groupby("state", sort=False)["last_name"].count()

state
DE      97
VA     433
SC     251
MD     305
PA    1053
      ... 
AK      16
PI      13
VI       4
GU       4
AS       2
Name: last_name, Length: 58, dtype: int64

### How Pandas GroupBy Works

In [14]:
#Before you get any further into the details, take a step back to look at .groupby() itself:

by_state = df.groupby("state")
print(by_state)

What is that DataFrameGroupBy thing? Its .__str__() doesn’t give you much information into what it actually is or how it works. The reason that a DataFrameGroupBy object can be difficult to wrap your head around is that it’s lazy in nature. It doesn’t really do any operations to produce a useful result until you say so.

One term that’s frequently used alongside .groupby() is split-apply-combine. This refers to a chain of three steps:

Split a table into groups
Apply some operations to each of those smaller tables
Combine the results
It can be difficult to inspect df.groupby("state") because it does virtually none of these things until you do something with the resulting object. Again, a Pandas GroupBy object is lazy. It delays virtually every part of the split-apply-combine process until you invoke a method on it.

So, how can you mentally separate the split, apply, and combine stages if you can’t see any of them happening in isolation? One useful way to inspect a Pandas GroupBy object and see the splitting in action is to iterate over it. This is implemented in DataFrameGroupBy.__iter__() and produces an iterator of (group, DataFrame) pairs for DataFrames:

In [15]:
for state, frame in by_state:
    print(f"First 2 entries for {state!r}")
    print("------------------------")
    print(frame.head(2), end="\n\n")

First 2 entries for 'AK'
------------------------
     last_name first_name   birthday gender type state        party
6617    Waskey      Frank 1875-04-20      M  rep    AK     Democrat
6645      Cale     Thomas 1848-09-17      M  rep    AK  Independent

First 2 entries for 'AL'
------------------------
    last_name first_name   birthday gender type state       party
911   Crowell       John 1780-09-18      M  rep    AL  Republican
990    Walker       John 1783-08-12      M  sen    AL  Republican

First 2 entries for 'AR'
------------------------
     last_name first_name   birthday gender type state party
1000     Bates      James 1788-08-25      M  rep    AR   NaN
1278    Conway      Henry 1793-03-18      M  rep    AR   NaN

First 2 entries for 'AS'
------------------------
          last_name first_name   birthday gender type state     party
10795         Sunia       Fofó 1937-03-13      M  rep    AS  Democrat
11752  Faleomavaega        Eni 1943-08-15      M  rep    AS  Democrat

F

   last_name  first_name   birthday gender type state                party
1      Bland  Theodorick 1742-03-21      M  rep    VA                  NaN
11   Grayson     William        NaT      M  sen    VA  Anti-Administration

First 2 entries for 'VI'
------------------------
      last_name first_name   birthday gender type state       party
10494     Evans     Melvin 1917-08-07      M  rep    VI  Republican
11086   de Lugo        Ron 1930-08-02      M  rep    VI    Democrat

First 2 entries for 'VT'
------------------------
   last_name first_name   birthday gender type state                party
41  Robinson      Moses 1741-03-22      M  sen    VT  Anti-Administration
86     Niles  Nathaniel 1741-04-03      M  rep    VT                  NaN

First 2 entries for 'WA'
------------------------
      last_name first_name   birthday gender type state     party
2977  Lancaster   Columbia 1803-08-26      M  rep    WA  Democrat
3050   Anderson      James 1822-02-16      M  rep    WA  Democra

If you’re working on a challenging aggregation problem, then iterating over the Pandas GroupBy object can be a great way to visualize the split part of split-apply-combine.

There are a few other methods and properties that let you look into the individual groups and their splits. The .groups attribute will give you a dictionary of {group name: group label} pairs. For example, by_state is a dict with states as keys. Here’s the value for the "PA" key:

In [17]:
by_state.groups["PA"]

Int64Index([    4,    19,    21,    27,    38,    57,    69,    76,    84,
               88,
            ...
            11838, 11862, 11871, 11873, 11883, 11887, 11926, 11938, 11952,
            11966],
           dtype='int64', length=1053)

Each value is a sequence of the index locations for the rows belonging to that particular group. In the output above, 4, 19, and 21 are the first indices in df at which the state equals “PA.”

You can also use .get_group() as a way to drill down to the sub-table from a single group:

In [18]:
by_state.get_group("PA")

last_name first_name   birthday gender type state                party
4        Clymer     George 1739-03-16      M  rep    PA                  NaN
19       Maclay    William 1737-07-20      M  sen    PA  Anti-Administration
21       Morris     Robert 1734-01-20      M  sen    PA   Pro-Administration
27      Wynkoop      Henry 1737-03-02      M  rep    PA                  NaN
38       Jacobs     Israel 1726-06-09      M  rep    PA                  NaN
...         ...        ...        ...    ...  ...   ...                  ...
11887     Brady     Robert 1945-04-07      M  rep    PA             Democrat
11926   Shuster       Bill 1961-01-10      M  rep    PA           Republican
11938   Rothfus      Keith 1962-04-25      M  rep    PA           Republican
11952  Costello       Ryan 1976-09-07      M  rep    PA           Republican
11966    Marino        Tom 1952-08-15      M  rep    PA           Republican

[1053 rows x 7 columns]

**This is virtually equivalent to using .loc[]. You could get the same output with something like df.loc[df["state"] == "PA"].**

Note: I use the generic term Pandas GroupBy object to refer to both a DataFrameGroupBy object or a SeriesGroupBy object, which have a lot of commonalities between them.

It’s also worth mentioning that .groupby() does do some, but not all, of the splitting work by building a Grouping class instance for each key that you pass. However, many of the methods of the BaseGrouper class that holds these groupings are called lazily rather than at __init__(), and many also use a cached property design.

Next, what about the apply part? You can think of this step of the process as applying the same operation (or callable) to every “sub-table” that is produced by the splitting stage. (I don’t know if “sub-table” is the technical term, but I haven’t found a better one 🤷‍♂️)

From the Pandas GroupBy object by_state, you can grab the initial U.S. state and DataFrame with next(). When you iterate over a Pandas GroupBy object, you’ll get pairs that you can unpack into two variables

In [19]:
state, frame = next(iter(by_state))  # First tuple from iterator
state

frame.head(3)


last_name first_name   birthday gender type state        party
6617    Waskey      Frank 1875-04-20      M  rep    AK     Democrat
6645      Cale     Thomas 1848-09-17      M  rep    AK  Independent
7440   Grigsby     George 1874-12-02      M  rep    AK          NaN

In [20]:
df.groupby("state")["last_name"].count()

state
AK     16
AL    209
AR    117
AS      2
AZ     49
     ... 
VT    115
WA     96
WI    198
WV    120
WY     40
Name: last_name, Length: 58, dtype: int64

The apply stage, when applied to your single, subsetted DataFrame, would look like this:

In [21]:
frame["last_name"].count()  # Count for state == 'AK'

16

## Example 2: Air Quality Dataset

The air quality dataset contains hourly readings from a gas sensor device in Italy. Missing values are denoted with -200 in the CSV file. You can use read_csv() to combine two columns into a timestamp while using a subset of the other columns:

In [22]:
#import pandas as pd

df = pd.read_csv(
    "airqual.csv",
    parse_dates=[["Date", "Time"]],
    na_values=[-200],
    usecols=["Date", "Time", "CO(GT)", "T", "RH", "AH"]
).rename(
    columns={
        "CO(GT)": "co",
        "Date_Time": "tstamp",
        "T": "temp_c",
        "RH": "rel_hum",
        "AH": "abs_hum",
    }
).set_index("tstamp")

ValueError: Passed header names mismatches usecols

## Example 3: News Aggregator Dataset

In [23]:
import datetime as dt
import pandas as pd

def parse_millisecond_timestamp(ts: int) -> dt.datetime:
    """Convert ms since Unix epoch to UTC datetime instance."""
    return dt.datetime.fromtimestamp(ts / 1000, tz=dt.timezone.utc)

df = pd.read_csv(
    "news.csv",
    sep="\t",
    header=None,
    index_col=0,
    names=["title", "url", "outlet", "category", "cluster", "host", "tstamp"],
    parse_dates=["tstamp"],
    date_parser=parse_millisecond_timestamp,
    dtype={
        "outlet": "category",
        "category": "category",
        "cluster": "category",
        "host": "category",
    },
)

To read it into memory with the proper dyptes, you need a helper function to parse the timestamp column. This is because it’s expressed as the number of milliseconds since the Unix epoch, rather than fractional seconds, which is the convention. Similar to what you did before, you can use the Categorical dtype to efficiently encode columns that have a relatively small number of unique values relative to the column length.

Each row of the dataset contains the title, URL, publishing outlet’s name, and domain, as well as the publish timestamp. cluster is a random ID for the topic cluster to which an article belongs. category is the news category and contains the following options:

b for business
t for science and technology
e for entertainment
m for health

In [24]:
df.iloc[0]

title       Fed official says weak data caused by weather,...
url         http://www.latimes.com/business/money/la-fi-mo...
outlet                                      Los Angeles Times
category                                                    b
cluster                         ddUyU0VZz0BRneMioxUPQVP6sIxvM
host                                          www.latimes.com
tstamp                       2014-03-10 16:52:50.698000+00:00
Name: 1, dtype: object

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 422419 entries, 1 to 422937
Data columns (total 7 columns):
 #   Column    Non-Null Count   Dtype              
---  ------    --------------   -----              
 0   title     422419 non-null  object             
 1   url       422419 non-null  object             
 2   outlet    422417 non-null  category           
 3   category  422419 non-null  category           
 4   cluster   422419 non-null  category           
 5   host      422419 non-null  category           
 6   tstamp    422419 non-null  datetime64[ns, UTC]
dtypes: category(4), datetime64[ns, UTC](1), object(2)
memory usage: 16.9+ MB


In [26]:
df.head()

title                                                url             outlet category                        cluster                 host                           tstamp
1  Fed official says weak data caused by weather,...  http://www.latimes.com/business/money/la-fi-mo...  Los Angeles Times        b  ddUyU0VZz0BRneMioxUPQVP6sIxvM      www.latimes.com 2014-03-10 16:52:50.698000+00:00
2  Fed's Charles Plosser sees high bar for change...  http://www.livemint.com/Politics/H2EvwJSK2VE6O...           Livemint        b  ddUyU0VZz0BRneMioxUPQVP6sIxvM     www.livemint.com 2014-03-10 16:52:51.207000+00:00
3  US open: Stocks fall after Fed official hints ...  http://www.ifamagazine.com/news/us-open-stocks...       IFA Magazine        b  ddUyU0VZz0BRneMioxUPQVP6sIxvM  www.ifamagazine.com 2014-03-10 16:52:51.550000+00:00
4  Fed risks falling 'behind the curve', Charles ...  http://www.ifamagazine.com/news/fed-risks-fall...       IFA Magazine        b  ddUyU0VZz0BRneMioxUPQVP6sIxvM  www.ifamagazine.com 2014-03-10 16:52:51.793000+00:00
5  Fed's Plosser: Nasty Weather Has Curbed Job Gr...  http://www.moneynews.com/Economy/federal-reser...          Moneynews        b  ddUyU0VZz0BRneMioxUPQVP6sIxvM    www.moneynews.com 2014-03-10 16:52:52.027000+00:00

In [28]:
df.category.value_counts()

e    152469
b    115967
t    108344
m     45639
Name: category, dtype: int64

### Using Lambda Functions in .groupby()

This dataset invites a lot more potentially involved questions. I’ll throw a random but meaningful one out there: which outlets talk most about the Federal Reserve? Let’s assume for simplicity that this entails searching for case-sensitive mentions of "Fed". Bear in mind that this may generate some false positives with terms like “Federal Government.”

To count mentions by outlet, you can call .groupby() on the outlet, and then quite literally .apply() a function on each group:

In [29]:
df.groupby("outlet", sort=False)["title"].apply(
    lambda ser: ser.str.contains("Fed").sum()
).nlargest(10)

outlet
Reuters                         161
NASDAQ                          103
Businessweek                     93
Investing.com                    66
Wall Street Journal \(blog\)     61
MarketWatch                      56
Moneynews                        55
Bloomberg                        53
GlobalPost                       51
Economic Times                   44
Name: title, dtype: int64

### Improving the Performance of .groupby()

Let’s backtrack again to .groupby(...).apply() to see why this pattern can be suboptimal. To get some background information, check out How to Speed Up Your Pandas Projects. What may happen with .apply() is that it will effectively perform a Python loop over each group. While the .groupby(...).apply() pattern can provide some flexibility, it can also inhibit Pandas from otherwise using its Cython-based optimizations.

All that is to say that whenever you find yourself thinking about using .apply(), ask yourself if there’s a way to express the operation in a vectorized way. In that case, you can take advantage of the fact that .groupby() accepts not just one or more column names, but also many array-like structures:

A 1-dimensional NumPy array
A list
A Pandas Series or Index
Also note that .groupby() is a valid instance method for a Series, not just a DataFrame, so you can essentially inverse the splitting logic. With that in mind, you can first construct a Series of Booleans that indicate whether or not the title contains "Fed":

In [30]:
mentions_fed = df["title"].str.contains("Fed")
type(mentions_fed)


pandas.core.series.Series

Now, .groupby() is also a method of Series, so you can group one Series on another:

In [31]:
import numpy as np
mentions_fed.groupby(
    df["outlet"], sort=False
).sum().nlargest(10).astype(np.uintc)



outlet
Reuters                         161
NASDAQ                          103
Businessweek                     93
Investing.com                    66
Wall Street Journal \(blog\)     61
MarketWatch                      56
Moneynews                        55
Bloomberg                        53
GlobalPost                       51
Economic Times                   44
Name: title, dtype: uint32

The two Series don’t need to be columns of the same DataFrame object. They just need to be of the same shape:

In [32]:
mentions_fed.shape

df["outlet"].shape

(422419,)

Finally, you can cast the result back to an unsigned integer with np.uintc if you’re determined to get the most compact result possible. Here’s a head-to-head comparison of the two versions that will produce the same result:

In [33]:
df.groupby("outlet", sort=False)["title"].apply(
    lambda ser: ser.str.contains("Fed").sum()
).nlargest(10)

outlet
Reuters                         161
NASDAQ                          103
Businessweek                     93
Investing.com                    66
Wall Street Journal \(blog\)     61
MarketWatch                      56
Moneynews                        55
Bloomberg                        53
GlobalPost                       51
Economic Times                   44
Name: title, dtype: int64

In [34]:
# Version 2: using vectorization
mentions_fed.groupby(
    df["outlet"], sort=False
).sum().nlargest(10).astype(np.uintc)

outlet
Reuters                         161
NASDAQ                          103
Businessweek                     93
Investing.com                    66
Wall Street Journal \(blog\)     61
MarketWatch                      56
Moneynews                        55
Bloomberg                        53
GlobalPost                       51
Economic Times                   44
Name: title, dtype: uint32

On my laptop, Version 1 takes 4.01 seconds, while Version 2 takes just 292 milliseconds. This is an impressive 14x difference in CPU time for a few hundred thousand rows. Consider how dramatic the difference becomes when your dataset grows to a few million rows!

## Pandas GroupBy: Putting It All Together

If you call dir() on a Pandas GroupBy object, then you’ll see enough methods there to make your head spin! It can be hard to keep track of all of the functionality of a Pandas GroupBy object. One way to clear the fog is to compartmentalize the different methods into what they do and how they behave.

Broadly, methods of a Pandas GroupBy object fall into a handful of categories:

Aggregation methods (also called reduction methods) “smush” many data points into an aggregated statistic about those data points. An example is to take the sum, mean, or median of 10 numbers, where the result is just a single number.

Filter methods come back to you with a subset of the original DataFrame. This most commonly means using .filter() to drop entire groups based on some comparative statistic about that group and its sub-table. It also makes sense to include under this definition a number of methods that exclude particular rows from each group.

Transformation methods return a DataFrame with the same shape and indices as the original, but with different values. With both aggregation and filter methods, the resulting DataFrame will commonly be smaller in size than the input DataFrame. This is not true of a transformation, which transforms individual values themselves but retains the shape of the original DataFrame.

Meta methods are less concerned with the original object on which you called .groupby(), and more focused on giving you high-level information such as the number of groups and indices of those groups.

Plotting methods mimic the API of plotting for a Pandas Series or DataFrame, but typically break the output into multiple subplots.